In [1]:
import os
import pandas as pd
from relm.histogram import Histogram
from relm.mechanisms import SmallDB
import numpy as np
from itertools import product
import scipy.sparse as sps

In [36]:
fp = '20200811_QLD_dummy_dataset_individual_v2.xlsx'
df = pd.read_excel(fp)
df.drop(["NOTF_ID", "LGA", "HHS"] + list(df.columns[12:]), axis=1, inplace=True)

hist = Histogram(df)
real_database = hist.get_db()

In [37]:
from itertools import product
queries = []

cols = ["AGEGRP5"]
vals = product(*[list(hist.column_sets[hist.column_dict[c]]) for c in cols])
queries.extend(dict(zip(cols, val)) for val in vals)

queries = sps.vstack([hist.get_query_vector(q) for q in queries])

In [38]:
smalldb = SmallDB(epsilon=4.0, data=real_database, alpha=0.001)

In [39]:
%time synthetic_database = smalldb.release(queries)

CPU times: user 4.32 s, sys: 502 ms, total: 4.83 s
Wall time: 4.83 s


In [42]:
print(real_database.shape)
print(synthetic_database.shape)
print(queries.shape)

(28554240,)
(28554240,)
(10, 28554240)


In [45]:
real_responses = (queries @ real_database)/real_database.sum()
synthetic_responses = (queries @ synthetic_database) / synthetic_database.sum()

In [46]:
print(real_responses)
print(synthetic_responses)

[0.0113 0.0103 0.2348 0.1523 0.1408 0.1436 0.1028 0.0889 0.0955 0.0197]
[0.10003129 0.09983509 0.10008959 0.10009699 0.10006889 0.10003929
 0.09992659 0.09997089 0.10002019 0.09992119]
